# Imports

In [ ]:
import numpy as np
import glob
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
from eval_utils import get_output_files, generate_results_df_from_output_files, transform_name_to_label, generate_thresholding_df_from_output_files
import matplotlib.pyplot as plt
import math

%load_ext autoreload
%autoreload 2

# Parsing
accumulate output files

In [ ]:
corruptions_standard = [
    'gaussian_noise', 'impulse_noise', 'shot_noise', 'defocus_blur', 'glass_blur', 'zoom_blur', 'motion_blur',
    'snow', 'frost', 'brightness', 'elastic_transform', 'jpeg_compression', 'contrast', 'pixelate', 'fog'   
]

corruptions_holdout = ['gaussian_blur', 'saturate', 'speckle_noise', 'spatter']

severities = [1,2,3,4,5]

methods = ['source', 'tent', 'norm', 'gce']

n_ood_samples = [100,200]

ood_datasets = ["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]

ood_settings = list()
for dataset in ood_datasets:
    for number in n_ood_samples:
        ood_settings.append(dataset + "_" + str(number))
ood_settings

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/holdout/"))
#files = get_output_files(output_dir="./04_output/output_leo/01_verify_evis_results/*_final/")
#files.extend(get_output_files(output_dir="./04_output/output_leo/02*/"))
#files.extend(get_output_files(output_dir="./04_output/output_leo/04*/"))
#files.extend(get_output_files(output_dir="./04_output/output_leo/05*/"))
#files.extend(get_output_files(output_dir="./04_output/output_leo/06*/"))
#files.extend(get_output_files(output_dir="./04_output/output_leo/08*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/08*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/09*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/10*/*/"))
#files.extend(get_output_files(output_dir="./04_output/output_leo/10*/*/"))

print((len(files)))

In [ ]:
results_df = generate_results_df_from_output_files(files)
results_df[results_df["Adaptation"]=="norm"]

In [ ]:
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
results_df

### Agg results
transform them into human readable df

In [ ]:
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"}) 
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)

pd.set_option('display.max_rows', 195)
agg_results

### Table1: All Methods, All Learning Rates, All nOODSamples, No oodDetection

In [ ]:
noods = [0,100,200]
filtered_df_1 = agg_results.query("nOOD in @noods and PL_Threshold not in ['0.5','0.7'] and OOD_DetMethod == 'none'")

In [ ]:
filtered_df_1[("ErrorRate", "min")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].min(axis=1)
filtered_df_1[("ErrorRate", "minEpoch")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].idxmin(axis=1)
filtered_df_1

#### best

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    new_key = row[:5]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = filtered_df_1[("ErrorRate", "min")].loc[row]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = filtered_df_1[("ErrorRate", "min")].loc[row]

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]
reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

# nur die beste lr
df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]


df_1_2


In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_200_all_lr_best")

#### all

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]
reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1[reordercolumns]
df_1_2 = df_1_2.reindex(reorderlist, level= "Adaptation")


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_200_all_lr")

#### origin

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    new_key = row[:5]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1]]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1]]

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]
reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]


df_1_2

In [ ]:
for row in df_1_2.index:
    print(row)
    if row != "source":
        for column in df_1_2.columns:
            temp = df_1_2.loc[row,column]
            print("'{}': [{},0.0,'none',0.0,{}],".format(column, temp[1], temp[2]))

#### standard corruptions

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/12*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/standard/"))
results_df = generate_results_df_from_output_files(files)
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
dict_of_results = dict()

for row in agg_results.index:
    adaptation = row[0]
    lr = row[1]
    ood_threshold = row[4]
    temp_series = agg_results.loc[row]
    score = temp_series[temp_series.notna()][-1]
    new_key = row[0]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][agg_results.index.names[i]+"-"+str(row[i])] = score
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = score


In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation"]#,"lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_200_standard_best")

### Table2: ood detection

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/10*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/holdout/"))
results_df = generate_results_df_from_output_files(files)
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
noods = [0,100]
filtered_df_1 = agg_results.query("nOOD in @noods and PL_Threshold not in ['0.5','0.7']")
filtered_df_1[("ErrorRate", "min")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].min(axis=1)
filtered_df_1[("ErrorRate", "minEpoch")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].idxmin(axis=1)


#### best

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    new_key = row[:5]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = filtered_df_1[("ErrorRate", "min")].loc[row]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = filtered_df_1[("ErrorRate", "min")].loc[row]
dict_of_results

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]#["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD","CIFAR100-100", "CIFAR100C-100", "SVHN-100", "SVHNC-100"]#["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

# nur die beste lr
df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_ood_detection_best")

#### alle

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["CIFAR100-100", "CIFAR100C-100", "SVHN-100", "SVHNC-100"]#["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

# alle

df_1_2 = df_1_1[reordercolumns]
df_1_2 = df_1_2.reindex(reorderlist, level= "Adaptation")


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_ood_detection_all")

#### origin

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    new_key = row[:5]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1], row[4]]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1]]

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["softpl", "hardpl", "tent", "gce"]
reordercolumns = ["CIFAR100-100", "CIFAR100C-100", "SVHN-100", "SVHNC-100"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1[reordercolumns]
df_1_2 = df_1_2.reindex(reorderlist, level= "Adaptation")


df_1_2

In [ ]:
for row in df_1_2.index:
    print(row)
    if row != "source":
        for column in df_1_2.columns:
            
            temp = df_1_2.loc[row,column]
            if isinstance(temp,list):
                print("'{}': [{},0.0,'none',0.0,{},{}],".format(column, temp[1], temp[2],temp[3]))

#### standard corruptions

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/13*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/standard/"))
results_df = generate_results_df_from_output_files(files)
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
dict_of_results = dict()

for row in agg_results.index:
    adaptation = row[0]
    lr = row[1]
    ood_threshold = row[4]
    temp_series = agg_results.loc[row]
    score = temp_series[temp_series.notna()][-1]
    new_key = row[0]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][agg_results.index.names[i]+"-"+str(row[i])] = score
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = score
    

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation"]#,"lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD", "CIFAR100-100", "CIFAR100C-100", "SVHN-100", "SVHNC-100"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_100_ood_detection_standard_best")

### Table3: thresholding w/o ood data

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/11*/*/"))
results_df = generate_results_df_from_output_files(files)
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
agg_results[("ErrorRate", "min")] = agg_results[[("ErrorRate", str(x))for x in range(1,7)]].min(axis=1)
agg_results[("ErrorRate", "minEpoch")] = agg_results[[("ErrorRate", str(x))for x in range(1,7)]].idxmin(axis=1)
filtered_df_1 = agg_results.copy()

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    
    new_key = row[:5]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = filtered_df_1[("ErrorRate", "min")].loc[row]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = filtered_df_1[("ErrorRate", "min")].loc[row]

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD"]#["noOOD", "CIFAR100-100", "CIFAR100-200", "CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200", "SVHNC-100", "SVHNC-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

# alle

df_1_2 = df_1_1[reordercolumns]
df_1_2 = df_1_2.reindex(reorderlist, level= "Adaptation")


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_thresholding_noood")

#### origin

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    
    new_key = row[:5]
   
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1]]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = [filtered_df_1[("ErrorRate", "min")].loc[row], row[1], filtered_df_1[("ErrorRate", "minEpoch")].loc[row][1]]

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["source", "norm","softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1[reordercolumns]
df_1_2 = df_1_2.reindex(reorderlist, level= "Adaptation")


df_1_2

#### standard corruptions

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/14*/*/"))
results_df = generate_results_df_from_output_files(files)
results_df["nOOD"] = results_df[["CIFAR100", "CIFAR100C", "SVHN", "SVHNC"]].sum(axis=1)
agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType','nOOD'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
dict_of_results = dict()

for row in agg_results.index:
    adaptation = row[0]
    lr = row[1]
    ood_threshold = row[4]
    temp_series = agg_results.loc[row]
    score = temp_series[temp_series.notna()][-1]
    new_key = row[0]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][agg_results.index.names[i]+"-"+str(row[i])] = score
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = score

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation"]#,"lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["softpl", "hardpl", "tent", "gce"]
reordercolumns = ["noOOD"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]

df_1_2


In [ ]:
df_1_2.to_csv("08_results_and_figures/res_thresholding_noood_standard")

### ablation on q

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/holdout/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/08*/*/"))

results_df = generate_results_df_from_output_files(files)

agg_results = pd.pivot_table(results_df, values=["Error"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType'], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"}) #, "count": "# of entries"
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
agg_results

In [ ]:
filtered_df_1 = agg_results.query("Adaptation=='gce' and CIFAR100 == 0")
filtered_df_1[("ErrorRate", "min")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].min(axis=1)
filtered_df_1[("ErrorRate", "minEpoch")] = filtered_df_1[[("ErrorRate", str(x))for x in range(1,7)]].idxmin(axis=1)
filtered_df_1

In [ ]:
dict_of_results = dict()
for row in filtered_df_1[("ErrorRate", "min")].index:
    
    new_key = row[:5]
    
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][filtered_df_1.index.names[i]+"-"+str(row[i])] = filtered_df_1[("ErrorRate", "min")].loc[row]
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = filtered_df_1[("ErrorRate", "min")].loc[row]
    

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation","lr","PL_Threshold","oodMethod","oodThreshold"]
reordercolumns = ["noOOD","CIFAR100C-100", "CIFAR100C-200", "SVHN-100", "SVHN-200"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

# nur die beste lr
df_1_2 = df_1_1.min(level= "PL_Threshold")
df_1_2 = df_1_2[reordercolumns]


df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/ablation_q")

### which data is filtered out?

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/13*/*/"))
results_df = generate_thresholding_df_from_output_files(files)
results_df

In [ ]:
agg_results = pd.pivot_table(results_df, values=["Error","FilteredTotal", "FilteredCifar", "FilteredOOD"], index=['Adaptation', 'LearningRate', "PL_Threshold", "OOD_DetMethod", "OOD_Threshold", 'CIFAR100', 'CIFAR100C', 'SVHN', 'SVHNC', 'CorruptionType'], columns=["Epoch"], aggfunc={np.mean})

agg_results = agg_results.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
dict_of_results = dict()

for row in agg_results.index:
    adaptation = row[0]
    lr = row[1]
    ood_threshold = row[4]
    temp_series = agg_results.loc[row]
    
    score = []
    for i in ["Error", "FilteredCifar", "FilteredOOD"]:
        
        score.append(temp_series[(i,"mean")][temp_series[(i,"mean")].notna()][-1])
    new_key = row[0]
    if new_key not in dict_of_results.keys():
        dict_of_results[new_key] = dict()
    used = False
    for i in range(5,9):
        if row[i] != 0:
            dict_of_results[new_key][agg_results.index.names[i]+"-"+str(row[i])] = score
            used=True
    if used==False:
        dict_of_results[new_key]["noOOD"] = score

dict_of_results

In [ ]:
df_1_1 = pd.DataFrame(dict_of_results).T
df_1_1.index.names = ["Adaptation"]#,"lr","PL_Threshold","oodMethod","oodThreshold"]

reorderlist = ["softpl", "hardpl", "tent", "gce"]
reordercolumns = ["CIFAR100-100", "CIFAR100C-100", "SVHN-100", "SVHNC-100"] #["noOOD", "CIFAR100-200", "CIFAR100C-200", "SVHN-200", "SVHNC-200"]

df_1_2 = df_1_1.min(level= "Adaptation").reindex(reorderlist)
df_1_2 = df_1_2[reordercolumns]

df_1_2

In [ ]:
df_1_2.to_csv("08_results_and_figures/res_thresholding_filtered_out")

## Graphs

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/09*/*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/10*/*/"))
results_df = generate_results_df_from_output_files(files)
data = results_df.query("LearningRate not in ['0.003', '0.0003'] and CIFAR100==0 and CIFAR100C==0 and SVHNC==0")
data = data.drop_duplicates()
data

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))


for setting in ["none", "threshold_n_images"]:

    filtered_data = data.query("Adaptation == @adaptation and OOD_DetMethod == @setting")
    source_data = data.query("Adaptation == 'source'")
    norm_data = data.query("Adaptation == 'norm'")

    data_list = [filtered_data, norm_data]#[filtered_data, source_data, norm_data]

    epochs = range(1,7)
    
    for data_elem in data_list:
        max = list()
        min = list()
        mean = list()
        for epoch in epochs:
            
            mean.append(data_elem[data_elem["Epoch"]==str(epoch)]["Error"].mean())
            
        ax.plot(epochs, mean, label=adaptation+'_'+setting)#label="PL_Threshold = {}".format(condition_temp)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Error Rate [%]")    


fig.subplots_adjust(wspace=0.35, hspace=0.35)
fig.patch.set_facecolor('white')
fig.legend(#title = "[Adaptation, lr, Corruptions, OOD_samples, OOD_detection]", 
    loc="upper center", bbox_to_anchor=(0.5,0))
plt.show()

In [ ]:
data = filtered_df

## accuracy of source on the corruptions
source_performance_dict = {
    "holdout": 23.42,
    "standard": 26.54
}

fig, ax = plt.subplots(figsize=(20,5))

plot_source_as_0 = False
plot_source_as_hline = False
plot_min_max_bands = True

if plot_source_as_0:
    n_epochs = len(data["ErrorRate"].columns)
    epochs = range(0,n_epochs+1)
    x = epochs

    for row in range(len(data)):
        y = [source_performance_dict[data.iloc[row].name[9]]]
        y.extend(data.iloc[row]["ErrorRate"].tolist())
        ax.plot(x, y, label=transform_name_to_label(data.iloc[row].name))

elif plot_source_as_hline:
    n_epochs = len(data["ErrorRate"].columns)
    epochs = range(1,n_epochs+1)
    x = epochs

    for row in range(len(data)):
        y = data.iloc[row]["ErrorRate"].tolist()
        ax.hlines(y=source_performance_dict[data.iloc[row].name[9]],xmin=1, xmax=6)
        ax.plot(x, y, label=transform_name_to_label(data.iloc[row].name))

elif plot_min_max_bands:
    n_epochs = 6#len(data["Error"].columns)
    epochs = range(1,n_epochs+1)
    x = epochs
    #print(data)
    condition_temps = ["none","threshold_n_images"]#["norm","softpl", "harpl","tent","gce"]#['0.7','0.8']
    condition = "OOD_DetMethod == @condition_temp"#"PL_Threshold == @condition_temp"
    for condition_temp in condition_temps:
        data_part = data.query(condition)
        #print(data_part)
        max = data_part["Error"].max().tolist()
        min = data_part["Error"].min().tolist()
        mean = data_part["Error"].mean().tolist()
        (print(max,min,mean))
        ax.plot(x, mean, label="PL_Threshold = {}".format(condition_temp))
        ax.fill_between(x, max, min, alpha=0.2)

    fig.legend(#title = "[Adaptation, lr, Corruptions, OOD_samples, OOD_detection]", 
                loc="upper center", bbox_to_anchor=(0.5,0))#len(data)*-0.04))

labels = [str(epoch) for epoch in epochs]
ax.set_xticks(ticks=x, labels = labels)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.set_ylim(0,80)

plt.xlabel("Epochs")
plt.ylabel("Error Rate [%]")
plt.show()

## chart error rate per corruption and severity

In [ ]:
files = []
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/standard*/"))
files.extend(get_output_files(output_dir="./04_output/output_leo/31*/holdout*/"))
results_df = generate_results_df_from_output_files(files)
agg_results = pd.pivot_table(results_df, values=["Error"], index=["Adaptation", "Corruption", "Severity",], columns=["Epoch"], aggfunc={np.mean, "count"})["Error"]
agg_results = agg_results.rename(columns={"mean": "ErrorRate"})
agg_results.ErrorRate = agg_results.ErrorRate.round(decimals=1)
pd.set_option('display.max_rows', 195)
agg_results

In [ ]:
filtered_df_1 = agg_results.query("Adaptation=='source'")
data = filtered_df_1[("ErrorRate","1")].loc["source"]
data

In [ ]:
noise = ['gaussian_noise', 'shot_noise', 'impulse_noise', 'speckle_noise']
blur = ['defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur', 'gaussian_blur']
weather =['snow', 'frost', 'fog', 'brightness', 'spatter']
digital = ['contrast', 'elastic_transform', 'pixelate', 'jpeg_compression', 'saturate']
labels = noise+blur+weather+digital

noise1 = ['Gaussian noise', 'Shot noise', 'Impulse noise', '(Speckle noise)']
blur1 = ['Defocus blur', 'Glass blur', 'Motion blur', 'Zoom blur', '(Gaussian blur)']
weather1 =['Snow', 'Frost', 'Fog', 'Brightness', '(Spatter)']
digital1 = ['Contrast', 'Elastic transform', 'Pixelate', 'Jpeg compression', '(Saturate)']
labels_printable = noise1+blur1+weather1+digital1

data = data.reindex(labels, level="Corruption")
data

In [ ]:
data["fog"][1]

In [ ]:
ind = np.arange(len(labels)) 
severities = range(1,6)
width = 0.15

fig, ax = plt.subplots(figsize=(20,5))

values_per_severity = {new_list: [] for new_list in severities}
bars = list()
for severity in severities:
    for corruption in labels:
        values_per_severity[severity].append(data[corruption][severity])

    bar = plt.bar(ind+width*(severity-1), values_per_severity[severity], width, label=str(severity))
  
plt.xlabel("Corruptions")
plt.ylabel('Error Rate [%]')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
  
plt.xticks(ind+width,labels_printable, rotation=45)
plt.legend()
plt.rc('font', size=20) 
plt.show()

## tsne

### ours

In [ ]:
# mvp
file1 = "97_embeddings/"
file2 = "20220720_204411.npy"
abc = np.load(file1 +"x_"+ file2)
xyz = np.load(file1 +"y_"+ file2)
xyz = np.append(xyz, [10 for x in range(100)])
print(abc.shape, xyz.shape)
#abc = abc.reshape(abc.shape[0],640*64)
#print(abc.shape, abc1.shape, xyz.shape)
abc = abc.squeeze()
print(abc.shape)

# We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne = TSNE(n_components)
tsne_result = tsne.fit_transform(abc)
tsne_result.shape

tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': xyz})
fig, ax = plt.subplots(1)
scatter = plt.scatter(x = tsne_result[:,0], y = tsne_result[:,1], c = xyz)
#ax.legend(bbox_to_anchor=(1.05, 1))
legend1 = ax.legend(*scatter.legend_elements(),loc="lower left", title="Classes")
ax.add_artist(legend1)


In [ ]:
def get_tsne(file1, file2):
    x_embeddings = np.load(file1 +"x_"+ file2)
    labels = np.load(file1 +"y_"+ file2)
    #labels = np.append(labels, [10 for x in range(int(0.5*len(labels)))])
    labels = np.concatenate((labels[0:200], [10 for x in range(100)], 
                                labels[200:400], [10 for x in range(100)], 
                                labels[400:600], [10 for x in range(100)], 
                                labels[600:800], [10 for x in range(100)], 
                                labels[800:], [10 for x in range(100)]))

    x_embeddings_squeezed = x_embeddings.squeeze()
    print("Input shape: Embeddings: {}, Squeezed Embeddings: {}, labels: {}".format(x_embeddings.shape, x_embeddings_squeezed.shape, labels.shape))

    # We want to get TSNE embedding with 2 dimensions
    n_components = 2
    tsne = TSNE(n_components)
    tsne_result = tsne.fit_transform(x_embeddings_squeezed)
    print("tsne result shape: ", tsne_result.shape)

    return tsne_result, labels

def plot_tsne(tsne_result_dict, labels_dict):
    nrows = 3
    ncols = 2
    fig, axs = plt.subplots(nrows,ncols, figsize=(20,20))
    for row in range(nrows):
        for col in range(ncols):
            scatter = axs[row][col].scatter(x = tsne_result_dict[(row,col)][:,0], y = tsne_result_dict[(row,col)][:,1], c = labels_dict[(row,col)])
    legend1 = fig.legend(*scatter.legend_elements(),loc="lower left", title="Classes")
    fig.add_artist(legend1)

    cols = ["saturate", "speckle_noise"]
    rows = ['Epoch {}'.format(row) for row in range(1,4)]

    for ax, col in zip(axs[0], cols):
        ax.set_title(col)

    for ax, row in zip(axs[:,0], rows):
        ax.set_ylabel(row, rotation=0, size='large')

    fig.tight_layout()

def plot_tsne_1epoch(tsne_result_dict, labels_dict):
    nrows = 1
    ncols = 2
    fig, axs = plt.subplots(nrows,ncols, figsize=(20,10), squeeze=False)
    for row in range(nrows):
        for col in range(ncols):
            scatter = axs[row][col].scatter(x = tsne_result_dict[(row,col)][:,0], y = tsne_result_dict[(row,col)][:,1], c = labels_dict[(row,col)])
    legend1 = fig.legend(*scatter.legend_elements(),loc="lower left", title="Classes")
    fig.add_artist(legend1)

    cols = ["saturate", "speckle_noise"]
    rows = ['Epoch {}'.format(row) for row in range(1,4)]

    for ax, col in zip(axs[0], cols):
        ax.set_title(col)

    for ax, row in zip(axs[:,0], rows):
        ax.set_ylabel(row, rotation=0, size='large')

    fig.tight_layout()

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

file1 = "97_embeddings/tent_3epochs/"
tsne_result_dict = dict()
labels_dict = dict()

file2 = "tent_CIFAR100_100_22-07-29_115654_487096_satu.txt_115709.npy"
pos = (0,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_CIFAR100_100_22-07-29_115654_560074_spec.txt_115711.npy"
pos = (0,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_CIFAR100_100_22-07-29_115654_487096_satu.txt_115736.npy"
pos = (1,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_CIFAR100_100_22-07-29_115654_560074_spec.txt_115737.npy"
pos = (1,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_CIFAR100_100_22-07-29_115654_487096_satu.txt_115803.npy"
pos = (2,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_CIFAR100_100_22-07-29_115654_560074_spec.txt_115804.npy"
pos = (2,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

plot_tsne(tsne_result_dict, labels_dict)


In [ ]:
plot_tsne_1epoch(tsne_result_dict, labels_dict)

In [ ]:
file1 = "97_embeddings/tent_3epochs/"
tsne_result_dict = dict()
labels_dict = dict()

file2 = "tent_SVHNC_100_22-07-29_115654_842551_satu.txt_115716.npy"
pos = (0,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_SVHNC_100_22-07-29_115654_122570_spec.txt_115714.npy"
pos = (0,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_SVHNC_100_22-07-29_115654_842551_satu.txt_115959.npy"
pos = (1,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_SVHNC_100_22-07-29_115654_122570_spec.txt_120001.npy"
pos = (1,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_SVHNC_100_22-07-29_115654_842551_satu.txt_120242.npy"
pos = (2,0)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

file2 = "tent_SVHNC_100_22-07-29_115654_122570_spec.txt_120248.npy"
pos = (2,1)
tsne_result_dict[pos], labels_dict[pos] = get_tsne(file1, file2)

plot_tsne(tsne_result_dict, labels_dict)

In [ ]:
plot_tsne_1epoch(tsne_result_dict, labels_dict)

In [ ]:
file1 = "97_embeddings/tent_1epoch/"
file2 = "tent_CIFAR100_100_22-07-29_010433_326820_spec.txt_010517.npy"

get_tsne(file1, file2)


In [ ]:
file1 = "97_embeddings/tent_1epoch/"
file2 = "tent_CIFAR100C_100_22-07-29_010432_778817_satu.txt_010728.npy"

get_tsne(file1, file2)

In [ ]:
file1 = "97_embeddings/tent_1epoch/"
file2 = "tent_SVHN_100_22-07-29_010434_613801_satu.txt_010520.npy"

get_tsne(file1, file2)

In [ ]:
file1 = "97_embeddings/tent_1epoch/"
file2 = "tent_SVHNC_100_22-07-29_010434_479099_satu.txt_010740.npy"

get_tsne(file1, file2)

## More visualization: npy to png

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import PIL

def get_10_samples_per_label(dataset, classes, path, nImages=10, labels=range(100), string_dataset="cifar10"):

    nClasses = len(classes)
    fig, axs = plt.subplots(nClasses,nImages,figsize=(nImages*2, nClasses*2)) 
    counter = [0 for x in range(nClasses)]
    print(counter)
    
    total_images_searched = 15*len(labels)

    for i in range(total_images_searched):
        for j in classes:
            idx_j = classes.index(j)
            img, label = dataset[i]
            if label == j and counter[idx_j] < nImages:
                #print(j, counter[j])
                if string_dataset=="svhn":
                    axs[idx_j, counter[idx_j]].imshow(img.permute(1,2,0))
                    save_image(img, "{}/{}_{}.jpg".format(path, labels[j], counter[idx_j]))
                else:
                    axs[idx_j, counter[idx_j]].imshow(img)
                    img.save("{}/{}_{}.jpg".format(path, labels[j], counter[idx_j]))
                axs[idx_j, counter[idx_j]].axis("off")
                #save_image(img, "img{}_{}.png".format(j,counter[j]))
                
                counter[idx_j] += 1

    
    selected_labels = [labels[x] for x in classes]
    for ax, row in zip(axs[:,0], selected_labels):
        if string_dataset=="svhn":
            ax.annotate(row, xy=(0, 0), xytext=(-0.25, 0.5), size='xx-large',  xycoords="axes fraction")
        elif string_dataset=="cifar100":
            ax.annotate(row, xy=(0, 0), xytext=(-1.25, 0.5), size='xx-large',  xycoords="axes fraction")
        else:
            ax.annotate(row, xy=(0, 0), xytext=(-1, 0.5), size='xx-large',  xycoords="axes fraction")
    
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    fig.patch.set_facecolor('white')
    plt.show()
    print(counter)

def get_corrupted_samples(path, corruptions, classes, labels_txt, corruption_labels, severity=3, svhn=False):

    labels = np.load(path+"/labels.npy")

    fig, axs = plt.subplots(len(classes),len(corruptions),figsize=(len(corruptions)*2, len(classes)*2))

    for i, c in enumerate(corruptions):
        dataset = np.load(path+"/"+c+".npy")
        for j, l in enumerate(classes):
            idx = np.where(labels==l)[0][0]
        
            
            if svhn==True:
                img = dataset[severity*26032 + idx]
                img = np.reshape(img, (32,32,3))
            else:
                img = dataset[severity*10000 + idx]
            axs[j, i].imshow(img)
            axs[j, i].axis("off")
           
    selected_labels = [labels_txt[x] for x in classes]
    for ax, row in zip(axs[:,0], selected_labels):
        if svhn:
            ax.annotate(row, xy=(0, 0), xytext=(-0.25, 0.5), size='xx-large',  xycoords="axes fraction")
        else:
            ax.annotate(row, xy=(0, 0), xytext=(-1, 0.5), size='xx-large',  xycoords="axes fraction")

    for ax, col in zip(axs[0], corruption_labels):
        ax.annotate(col, xy=(0.5, 1), xytext=(0.5, 1.1), size='xx-large',  xycoords="axes fraction", ha="center",rotation=45)
    
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    fig.patch.set_facecolor('white')

    plt.show()
    

def get_severity_samples(path, corruptions, corruption_labels, severities=[1,2,3,4,5]):

    labels = np.load(path+"/labels.npy")

    fig, axs = plt.subplots(len(severities),len(corruptions),figsize=(len(corruptions)*2, len(severities)*2))

    for i, c in enumerate(corruptions):
        dataset = np.load(path+"/"+c+".npy")
        for j, l in enumerate(severities):
            idx = np.where(labels==5)[0][1]
            
            img = dataset[(l-1)*10000 + idx]
            axs[j, i].imshow(img)
            axs[j, i].axis("off")
           
    selected_labels = severities
    for ax, row in zip(axs[:,0], selected_labels):
        ax.annotate(row, xy=(0, 0), xytext=(-0.25, 0.5), size='xx-large',  xycoords="axes fraction")

    for ax, col in zip(axs[0], corruption_labels):
        ax.annotate(col, xy=(0.5, 1), xytext=(0.5, 1.1), size='xx-large',  xycoords="axes fraction", ha="center")
    
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    fig.patch.set_facecolor('white')
    
    plt.show()
    

#### cifar10

In [ ]:
path = "02_data/CIFAR-10"
cifar10 = datasets.CIFAR10(path, download=True, train=False)
labels = ["Airplane", "Automobile", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"]

get_10_samples_per_label(cifar10, classes=range(10), path=path, labels=labels)

#### cifar10c

In [ ]:
path = "02_data/CIFAR-10-C/CIFAR-10-C/"
noise = ['gaussian_noise', 'shot_noise', 'impulse_noise', 'speckle_noise']
blur = ['defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur', 'gaussian_blur']
weather =['snow', 'frost', 'fog', 'brightness', 'spatter']
digital = ['contrast', 'elastic_transform', 'pixelate', 'jpeg_compression', 'saturate']
labels = noise+blur+weather+digital

noise1 = ['Gaussian noise', 'Shot noise', 'Impulse noise', '(Speckle noise)']
blur1 = ['Defocus blur', 'Glass blur', 'Motion blur', 'Zoom blur', '(Gaussian blur)']
weather1 =['Snow', 'Frost', 'Fog', 'Brightness', '(Spatter)']
digital1 = ['Contrast', 'Elastic transform', 'Pixelate', 'Jpeg compression', '(Saturate)']
labels1 = noise1+blur1+weather1+digital1

corruptions = labels#["gaussian_blur", "saturate", "spatter", "speckle_noise"]
corruption_labels = labels1#["Gaussian blur", "Saturate", "Spatter", "Speckle noise"]
classes = range(10)#[0,2,5,9]#range(10) #[2,3,7,9]
labels = ["Airplane", "Automobile", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"]


get_corrupted_samples(path, corruptions, classes, labels, corruption_labels)


In [ ]:
path = "02_data/CIFAR-10-C/"
corruptions = ["gaussian_blur", "saturate", "spatter", "speckle_noise"]
corruption_labels = ["Gaussian blur", "Saturate", "Spatter", "Speckle noise"]

get_severity_samples(path, corruptions, corruption_labels, severities=[1,2,3,4,5])

#### cifar100

In [ ]:
path = "02_data/CIFAR-100"

labels = ["Apple", "Aquarium Fish", "Baby", "Bear", "Beaver", "Bed", "Bee", "Beetle", "Bicycle", "Bottle", "Bowl", "Boy", "Bridge", "Bus", "Butterfly", "Camel", "Can", "Castle", "Caterpillar", 
            "Cattle", "Chair", "Chimpanzee", "Clock", "Cloud", "Cockroach", "Couch", "Cra", "Crocodile", "Cup", "Dinosaur", "Dolphin", "Elephant", "Flatfish", "Forest", "Fox", "Girl", "Gamster", 
            "House", "Kangaroo", "Keyboard", "Lamp", "Lawn Mower", "Leopard", "Lion", "Lizard", "Lobster", "Man", "Maple Tree", "Motorcycle", "Mountain", "Mouse", "Mushroom", "Oak Tree", "Orange", 
            "Orchid", "Otter", "Palm Tree", "Pear", "Pickup Truck", "Pine Tree", "Plain", "Plate", "Poppy", "Porcupine", "Possum", "Rabbit", "Raccoon", "Ray", "Road", "Rocket", "Rose", "Sea", "Seal", 
            "Shark", "Shrew", "Skunk", "Skyscraper", "Snail", "Snake", "Spider", "Squirrel", "Streetcar", "Sunflower", "Sweet Pepper", "Table", "Tank", "Telephone", "Television", "Tiger", "Tractor", 
            "Train", "Trout", "Tulip", "Turtle", "Wardrobe", "Whale", "Willow Tree", "Wolf", "Woman", "Worm"]
labels.extend(range(10,100))
classes = range(80,100)#[0,3,49,99] #range(100)

cifar100 = datasets.CIFAR100(path, train=False, download=True)

get_10_samples_per_label(cifar100, classes=classes, path=path, labels=labels,nImages=3, string_dataset="cifar100")

### cifar100c

In [ ]:
path = "02_data/CIFAR-100-C"
corruptions = ["gaussian_blur", "saturate", "spatter", "speckle_noise"]
corruption_labels = ["Gaussian blur", "Saturate", "Spatter", "Speckle noise"]
classes = [0,3,49,99] #[0,3,8,9]#range(10) #[2,3,7,9]
labels = ["Apple", "Aquarium Fish", "Baby", "Bear", "Beaver", "Bed", "Bee", "Beetle", "Bicycle", "Bottle", "Bowl", "Boy", "Bridge", "Bus", "Butterfly", "Camel", "Can", "Castle", "Caterpillar", 
            "Cattle", "Chair", "Chimpanzee", "Clock", "Cloud", "Cockroach", "Couch", "Cra", "Crocodile", "Cup", "Dinosaur", "Dolphin", "Elephant", "Flatfish", "Forest", "Fox", "Girl", "Gamster", 
            "House", "Kangaroo", "Keyboard", "Lamp", "Lawn Mower", "Leopard", "Lion", "Lizard", "Lobster", "Man", "Maple Tree", "Motorcycle", "Mountain", "Mouse", "Mushroom", "Oak Tree", "Orange", 
            "Orchid", "Otter", "Palm Tree", "Pear", "Pickup Truck", "Pine Tree", "Plain", "Plate", "Poppy", "Porcupine", "Possum", "Rabbit", "Raccoon", "Ray", "Road", "Rocket", "Rose", "Sea", "Seal", 
            "Shark", "Shrew", "Skunk", "Skyscraper", "Snail", "Snake", "Spider", "Squirrel", "Streetcar", "Sunflower", "Sweet Pepper", "Table", "Tank", "Telephone", "Television", "Tiger", "Tractor", 
            "Train", "Trout", "Tulip", "Turtle", "Wardrobe", "Whale", "Willow Tree", "Wolf", "Woman", "Worm"]


get_corrupted_samples(path, corruptions, classes, labels, corruption_labels)


### svhn

In [ ]:
path = "02_data/SVHN"
svhn = datasets.SVHN(path, split="test", transform=transforms.ToTensor(), download=True)
classes = range(10)#[0,2,5,9] #range(10) #[0,1,5,9]
get_10_samples_per_label(dataset=svhn, classes=classes, path=path, string_dataset="svhn", nImages=10)

### svhnc

In [ ]:
path = "02_data/SVHN-C"
x_temp = np.load(path +"/saturate.npy")
labels = np.load(path+"/labels.npy")
img = x_temp[0]
img = np.reshape(img, (32,32,3))
plt.imshow(img)
print(labels[0])

In [ ]:
labels = np.load("02_data/SVHN-C"+"/labels.npy")
labels[:10]

In [ ]:
path = "02_data/SVHN-C"
corruptions = ["gaussian_blur", "saturate", "spatter", "speckle_noise"]
corruption_labels = ["Gaussian blur", "Saturate", "Spatter", "Speckle noise"]
classes = [0,2,5,9]#range(10) #[0,2,5,9]#range(10) #[0,1,5,9]#[2,3,7,9]

get_corrupted_samples(path=path, corruptions=corruptions, classes=classes, labels_txt=range(10), corruption_labels=corruption_labels, svhn=True)